In [18]:
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
import matplotlib.pyplot as plt 
from tqdm import tqdm

%matplotlib inline
# Lệnh %matplotlib inline là một magic command trong Jupyter Notebook (và các môi trường tương tự) cho phép 
# bạn hiển thị các biểu đồ và đồ thị được vẽ bằng thư viện Matplotlib trực tiếp trong notebook. 
# Khi bạn sử dụng lệnh này, các biểu đồ sẽ được nhúng ngay dưới ô (cell) mà bạn gọi lệnh vẽ.

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms


torch.manual_seed(1234)
np.random.seed(1234)
random.seed (1234)

# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

class ImageTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(resize, scale=(0.5,1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            
            'val': transforms.Compose([
                transforms.Resize(resize),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),

            'test': transforms.Compose([
                transforms.Resize(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
        }
    
    def __call__(self, img, phase='train'):
        return self.data_transform[phase](img)
        

        
def make_datapath_list(phase="train"):
    rootpath = "C:\\data\\hymenoptera_data\\"  # phải có 2 dấu \ nếu không python nó sẽ hiểu lầm \n
    target_path = osp.join(rootpath+phase+"\**\*.jpg")
    
    path_list = []
    
    for path in glob.glob(target_path):
        path_list.append(path)
    
    return path_list



train_list = make_datapath_list("train")
val_list = make_datapath_list("val")
# 2 list tương đương với 2 tệp ảnh dành cho 2 mode


class MyDataset(data.Dataset):
    def __init__(self, file_list, transform=None, phase="train"):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
        
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path) # mở ảnh
        
        img_transformed = self.transform(img, self.phase)
        
        if self.phase == "train":
            label = img_path[31:35]
        elif self.phase == "val":
            label = img_path[29:33]
        
        if label == "ants":
            label = 0
        elif label == "bees":
            label = 1
        
        return img_transformed, label
    
    



resize = 224 
mean = (0.485, 0.456, 0.406) 
std = (0.229, 0.224, 0.225)

train_dataset = MyDataset(train_list, transform=ImageTransform(resize, mean, std), phase="train")
val_dataset = MyDataset(val_list, transform=ImageTransform(resize, mean, std), phase="val")
# index = 0
# print(train_dataset.__len__())               
# link, img, label = train_dataset.__getitem__(index)
# print(img)
# print(link)
# print(img.shape)
# print(label)


batch_size = 4

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle=False)

dataloader_dict = {"train":train_dataloader, "val":val_dataloader}

# duyệt tập dữ liệu theo từng batch

# for inputs, labels in dataloader_dict["train"]:
#     print(inputs.size()) # inputs.shape
#     print (labels)


# Network




## Network

In [ ]:
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)
net.classifier [6] = nn.Linear(in_features=4096, out_features=2, bias = True)

# bật train mode
net = net.train()
print(net)

# Loss


In [8]:
criterior = nn.CrossEntropyLoss ()

# Optimizer

In [12]:
params_to_update = []

update_params_name = ["classifier.6.weight", "classifier.6.bias"]

for name, param in net.named_parameters():
    if name in update_params_name:
        param.requires_grad = True
        params_to_update.append(param)
        print (name)
    else:
        param.requires_grad = False

print(params_to_update)

In [13]:
optimizer = optim.SGD(params = params_to_update, lr=0.001, momentum=0.9)

In [25]:
def train_model(net, dataloader_dict, criterior, optimizer, num_epochs):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("device: ", device)

    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs))

        # move network to device(GPU/CPU)
        net.to(device)

        torch.backends.cudnn.benchmark = True

        for phase in ["train", "val"]:
            
            if phase == "train":
                net.train()
            else:
                net.eval()
                
            total_loss = 0.0
            epoch_corrects = 0
            
            if (epoch == 0) and (phase == "train"):
                continue
                
            for inputs, labels in tqdm(dataloader_dict[phase]):
                # move inputs, labels to GPU/GPU
                inputs = inputs.to(device)
                labels = labels.to(device)

                # set gradient of optimizer to be zero
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == "train"):
                    
                    outputs = net(inputs)
                    loss = criterior(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                        
                    total_loss += loss.item()*inputs.size(0)
                    epoch_corrects += torch.sum(preds==labels.data)
                    
            
            epoch_loss = total_loss / len(dataloader_dict[phase].dataset)
            epoch_accuracy = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
                    
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_accuracy))



In [ ]:
num_epochs = 4
train_model(net, dataloader_dict, criterior, optimizer, num_epochs)